In [ ]:
# This have to be executed before running the slides
# Monkey patch unittest to be able to run it in the notebook

import unittest

def apply_jupyter_patch():
    default_unittest_main = unittest.main

    def jupyter_unittest_main(**kwargs):
        if "argv" not in kwargs:
            kwargs["argv"] = ['ignored']
        kwargs["exit"] = False
        default_unittest_main(**kwargs)

    unittest.main = jupyter_unittest_main

apply_jupyter_patch()

Testing
=======

- Introduction
- Python `unittest` module
- Estimate tests' quality
- Continuous integration

# What is it?

- A task consisting of checking that the **program** is working as expected
- Manually written **tests** which can be automatically executed
- Different methodologies: Always and before anything else

<img src="img/ttd-workflow.svg" style="width:40%;margin-left:auto;margin-right:auto;padding:2em;">

- Harry J.W. Percival (2014). [Test-Driven Development with Python. O'Reilly](https://www.oreilly.com/library/view/test-driven-development-with/9781449365141/)

# Presenter Notes

- Test inject input to the program, and check output
- Answer valid or not
- For maintenance: Reproduce bug in a test, then fix it.


# Why testing?

- Validate the code to the specifications
- Find problems early
- Facilitates change
- Documentation
- Code quality

  - Better design
  - Simplifies integration


# Why not?

- Extra work
- Not perfect
- Extra maintenance

  - More difficult to refactoring
  - Maintain environments

- Delays integration

# Presenter Notes

- 30% percent of the time of a project
- Having the structure set-up for testing encourages writing tests.
- Then... let's talk about the structure :-)


# What kinds of tests?

- <span style="color:#ee5aa0">**Unit tests**</span>: Tests independant pieces of code
- <span style="color:#19bdcd">**Integration tests**</span>: Tests components together
- <span style="color:#1aac5b">**System tests**</span>: Tests a completely integrated application
- <span style="color:#b8b800">**Acceptance tests**</span>: Tests the application with selected users (can't be automated)

<img src="img/test-kind.svg" style="width:40%;margin-left:auto;margin-right:auto;padding:0em;">

# Presenter Notes

The test pyramid is a concept developed by Mike Cohn, described in his book "Succeeding with Agile"

- Unit tests (dev point of view, fast, low cost)
- Integration tests
- Functional tests (user point of view, but slow, and expensive)

- Cost: unit << integration << system
- Fast to execute: unit >> integration >> system


# Where to put the tests?

Separate tests from the source code:

- Run the test from the command line.
- Separate test code when distributing.
- [...](https://docs.python.org/3/library/unittest.html#organizing-test-code)

Folder structure:

- In a separate `test/` folder.
- In `test` sub-packages in each Python package/sub-package,
  so that tests remain close to the source code.
  Tests are installed with the package and can be run from the installation.
- A `test_*.py` for each module and script (an more if needed).
- Consider separating tests that are long to run from the others.


# Where to put the tests?

- `project`
  - `setup.py`
  - `run_tests.py`
  - `package/`
    - `__init__.py`
    - `module1.py`
    - `test/`
      - `__init__.py`
      - `test_module1.py`
    - `subpackage/`
      - `__init__.py`
      - `module1.py`
      - `module2.py`
      - `test/`
        - `__init__.py`
        - `test_module1.py`
        - `test_module2.py`

# `unittest` Python module

[unittest](https://docs.python.org/3/library/unittest.html) is the default Python module for testing.

It provides features to:

- Write tests
- Discover tests
- Run those tests

Other frameworks exists:

- [pytest](http://pytest.org/)

# Write and run tests

The classe `unittest.TestCase` is the base class for writting tests for
Python code.

The function `unittest.main()` provides a command line interface to
discover and run the tests.

In [ ]:
import unittest

class TestMyTestCase(unittest.TestCase):

    def test_my_test(self):
        # Code to test
        a = round(3.1415)
        # Expected result
        b = 3
        self.assertEqual(a, b, msg="")

if __name__ == "__main__":
    unittest.main()

# Assertion functions

- Argument(s) to compare/evaluate.
- An additional error message.

- `assertEqual(a, b)` checks that `a == b`
- `assertNotEqual(a, b)` checks that `a != b`
- `assertTrue(x)` checks that `bool(x) is True`
- `assertFalse(x)`checks that `bool(x) is False`
- `assertIs(a, b)` checks that `a is b`
- `assertIsNone(x)` checks that `x is None`
- `assertIn(a, b)` checks that `a in b`
- `assertIsInstance(a, b)` checks that `isinstance(a, b)`

There's more, see [unittest TestCase documentation](https://docs.python.org/3/library/unittest.html#unittest.TestCase>)
or [Numpy testing documentation](http://docs.scipy.org/doc/numpy/reference/routines.testing.html).

# Example

In [ ]:
def my_true_round(value):
    # Default python3 round uses the round-to-even method (bankers’ rounding)
    # Here we use the round-half-away-from-zero method
    rounded = (int(value + (0.5 if value > 0 else -0.5)))
    return rounded

In [ ]:
import unittest

class TestMyTrueRound(unittest.TestCase):

    def test_positive(self):
        self.assertEqual(my_true_round(1.3), 1)

    def test_negative(self):
        self.assertEqual(my_true_round(-1.3), -1)

    def test_halfway_even(self):
        self.assertEqual(my_true_round(2.5), 3)

    def test_negative_halfway_even(self):
        self.assertEqual(my_true_round(-2.5), -3)

    def test_returned_type(self):
        self.assertIsInstance(my_true_round(0.0), int)

if __name__ == "__main__":
    unittest.main(defaultTest="TestMyTrueRound")
    # unittest.main(verbosity=2, defaultTest="TestMyTrueRound")

# Run from command line arguments

Auto discover tests of the current path

Running a specific `TestCase`:

Running a specific test method:

# Fixture

Tests might need to share some common initialisation/finalisation (e.g., create a temporary directory).

This can be implemented in ``setUp`` and ``tearDown`` methods of ``TestCase``.
Those methods are called before and after each test.

In [ ]:
class TestCaseWithFixture(unittest.TestCase):

    def setUp(self):
        self.file = open("img/test-pyramid.png", "rb")
        print("open file")

    def tearDown(self):
        self.file.close()
        print("close file")

    def test1(self):
        foo = self.file.read()
        # do some processing on foo
        print("processing1")

    def test2(self):
        foo = self.file.read()
        # do some processing on foo
        print("processing2")

if __name__ == "__main__":
    unittest.main(defaultTest='TestCaseWithFixture')

# Testing exception

In [ ]:
class TestMyTrueRound(unittest.TestCase):

    def test_bad_types(self):
        try:
            my_true_round('2')
            self.fail()
        except TypeError:
            self.assertTrue(True)

    def test_bad_types__better_way(self):
        with self.assertRaises(TypeError):
            my_true_round('2')

if __name__ == "__main__":
    unittest.main(defaultTest='TestMyTrueRound')

`TestCase.assertRaisesRegexp` also checks the message of the exception.

# Parametric tests

Running the same test with multiple values

Problems:

- The first failure stops the test, remaining test values are not processed.
- There is no information on the value for which the test has failed.

In [ ]:
class TestMyTrueRound(unittest.TestCase):

    HALFWAY_TESTS = ((0.5, 1), (1.5, 2), (2.5, 3))

    def test_all(self):
        for value, expected in self.HALFWAY_TESTS:
            self.assertEqual(my_true_round(value), expected)

    def test_all__better_way(self):
        for value, expected in self.HALFWAY_TESTS:
            with self.subTest(value=value, expected=expected):
                self.assertEqual(my_true_round(value), expected)

if __name__ == "__main__":
    unittest.main(defaultTest='TestMyTrueRound')

# Chaining tests

How-to run tests from many ``TestCase`` and many files at once:

- Explicit:
    Full control, boilerplate code.

- Automatic:
    No control

- Mixing approach


The [TestSuite](https://docs.python.org/3/library/unittest.html#unittest.TestSuite) class aggregates test cases and test suites through:

- Allow to test specific use cases
- Full control of the test sequence
- But requires some boilerplate code

# Chaining tests example

In [ ]:
class TestCase1(unittest.TestCase):
    def test(self): pass
class TestCase2(unittest.TestCase):
    def test(self): pass
class TestCaseGui1(unittest.TestCase):
    def test(self): pass
class TestCaseGui2(unittest.TestCase):
    def test(self): pass

def suite_without_gui():
    loadTests = unittest.defaultTestLoader.loadTestsFromTestCase
    suite = unittest.TestSuite()
    suite.addTest(loadTests(TestCase1))
    suite.addTest(loadTests(TestCase2))
    return suite

def suite_with_gui():
    loadTests = unittest.defaultTestLoader.loadTestsFromTestCase
    suite = unittest.TestSuite()
    suite.addTest(suite_without_gui())
    suite.addTest(loadTests(TestCaseGui1))
    suite.addTest(loadTests(TestCaseGui2))
    return suite

if __name__ == "__main__":
    # unittest.main(defaultTest='suite_without_gui')
    unittest.main(defaultTest='suite_with_gui')

# Class fixture

In [ ]:
class TestSample(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        # Called before all the tests of this class
        pass

    @classmethod
    def tearDownClass(cls):
        # Called after all the tests of this class
        pass

# Module fixture


In [ ]:
def setUpModule():
    # Called before all the tests of this module
    pass

def tearDownModule():
    # Called after all the tests of this module
    pass

# Skipping tests

If tests requires a specific OS, device, library...

In [ ]:
import unittest, socket

def get_database_access():
    socks = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    socks.settimeout(0.300)
    try:
        socks.connect(("my_smart_database", 42))
    except:
        return None
    finally:
        socks.close()

@unittest.skipIf(get_database_access() is None, 'No database')
class TestMyTrueRoundUsingDatabase(unittest.TestCase):

    def setUp(self):
        if get_database_access() is None:
            self.skipTest('No database')

    def test1(self):
        if get_database_access() is None:
            self.skipTest('No database')
        ...

    @unittest.skipIf(get_database_access() is None, 'No database')
    def test2(self):
        pass

if __name__ == "__main__":
    unittest.main(defaultTest='TestMyTrueRoundUsingDatabase')

# Test numpy

Numpy provides modules for unittests. See the [Numpy testing documentation](http://docs.scipy.org/doc/numpy/reference/routines.testing.html).

In [ ]:
import numpy

class TestNumpyArray(unittest.TestCase):

    def setUp(self):
        self.data1 = numpy.array([1, 2, 3, 4, 5, 6, 7])
        self.data2 = numpy.array([1, 2, 3, 4, 5, 6, 7.00001])

    # def test_equal__cant_work(self):
    #     self.assertEqual(self.data1, self.data2)
    #     self.assertTrue((self.data1 == self.data2).all())

    def test_equal(self):
        self.assertTrue(numpy.allclose(self.data1, self.data2, atol=0.0001))

    def test_equal__even_better(self):
        numpy.testing.assert_allclose(self.data1, self.data2, atol=0.0001)

if __name__ == "__main__":
    unittest.main(defaultTest='TestNumpyArray')

# Test resources

How to handle test data?

Need to separate (possibly huge) test data from python package.

Download test data and store it in a temporary directory during the tests if not available.

Example: [silx.utils.ExternalResources](https://github.com/silx-kit/silx/blob/master/silx/utils/utilstest.py)

# QTest

For GUI based on `PyQt`, `PySide` it is possible to use Qt's [QTest](http://doc.qt.io/qt-5/qtest.html).

It provides the basic functionalities for GUI testing.
It allows to send keyboard and mouse events to widgets.

In [ ]:
from PyQt5 import Qt

qapp = Qt.QApplication.instance()
if Qt.QApplication.instance() is None:
    qapp = Qt.QApplication([])

In [ ]:
from PyQt5 import Qt

class MyApplication(Qt.QMainWindow):

    def __init__(self, parent=None):
        super(MyApplication, self).__init__(parent=parent)
        self.initGui()

    def initGui(self):
        self._inputLine = Qt.QLineEdit(self)
        self._processButton = Qt.QPushButton(self)
        self._processButton.setText("Round")
        self._processButton.clicked.connect(self.processing)
        self._resultWidget = Qt.QLabel(self)

        widget = Qt.QWidget()
        layout = Qt.QVBoxLayout(widget)
        layout.addWidget(self._inputLine)
        layout.addWidget(self._resultWidget)
        layout.addWidget(self._processButton)
        self.setCentralWidget(widget)

    def processing(self):
        text = self._inputLine.text()
        data = float(text)
        result = my_true_round(data)
        self._resultWidget.setText("%d" % result)

In [ ]:
from PyQt5.QtTest import QTest

class TestGui(unittest.TestCase):

    def test_type_and_process(self):
        app = MyApplication()
        QTest.qWaitForWindowExposed(app)
        QTest.keyClicks(app._inputLine, '2.5', delay=100)  # Wait 100ms
        QTest.mouseClick(app._processButton, Qt.Qt.LeftButton, pos=Qt.QPoint(1, 1))
        self.assertEqual(app._resultWidget.text(), "3")

if __name__ == "__main__":
    unittest.main(defaultTest='TestGui')

Tighly coupled with the code it tests.
It needs to know the widget's instance and hard coded position of mouse events.

# Estimate tests' quality

Using [`coverage`](https://coverage.readthedocs.org) to gather coverage statistics while running the tests (`pip install coverage`).

# Estimate tests' quality

Execute the tests and generate an output file per module with annotations per lines.

# Continuous integration

Automatically testing a software for each changes applied to the source code.

Benefits:

- Be aware of problems early
    - Before merging a change on the code
    - On third-party library update (sometimes before the release)
    - Reduce the cost in case of problem
- Improve contributions and team work

Costs:

- Set-up and maintenance
- Test needs to be automated


# Continuous integration

- [Travis-CI](https://travis-ci.org/) (Linux and MacOS), [AppVeyor](http://www.appveyor.com/) (Windows), gitlab-CI (https://gitlab.esrf.fr)...
- A `.yml` file to describing environment, build, installation, test process

<img src="img/ci-workflow.svg" style="width:40%;margin-left:auto;margin-right:auto;padding:0em;">

# Continuous integration: Configuration

Example of configuration with Travis

# Sum-up

- Testing is a real job
- The amount of work must not be under estimate. Designing good test is about 1/3 of resources a project
- Tests should be done early, to identify design problems, and to improve team work
- To be tested, an application have to be architectured in this way
- Continuous integration is particularly useful to prevent regressions, and contributions
- Aiming exhausting tests is not needed and utopic. A coverage of most cases is a very good start
- Next step: Continuous deployment.